<a href="https://colab.research.google.com/github/VihaShahh/AI_Intern/blob/main/Demo4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install qai-hub

In [ ]:
!qai-hub configure --api_token 0bb7132654e37c476ab0375cc94e60a29def126b

qai-hub configuration saved to /root/.qai_hub/client.ini
==================== /root/.qai_hub/client.ini ====================
[api]
api_token = 0bb7132654e37c476ab0375cc94e60a29def126b
api_url = https://app.aihub.qualcomm.com
web_url = https://app.aihub.qualcomm.com
verbose = True




In [ ]:
!qai-hub list-devices

+-------------------------------+------------+----------+---------+---------------------------------------+---------------------------------------------------------+
|             Device            |     OS     |  Vendor  |   Type  |                Chipset                |                      CLI Invocation                     |
+-------------------------------+------------+----------+---------+---------------------------------------+---------------------------------------------------------+
|    Google Pixel 3 (Family)    | Android 10 |  Google  |  Phone  |    qualcomm-snapdragon-845, sdm845    |    --device "Google Pixel 3 (Family)" --device-os 10    |
|         Google Pixel 3        | Android 10 |  Google  |  Phone  |    qualcomm-snapdragon-845, sdm845    |         --device "Google Pixel 3" --device-os 10        |
|        Google Pixel 3a        | Android 10 |  Google  |  Phone  |    qualcomm-snapdragon-670, sdm670    |        --device "Google Pixel 3a" --device-os 10        |
|   

In [ ]:
pip install "qai-hub[torch]"

In [16]:
import qai_hub as hub
import torch
from torchvision.models import mobilenet_v2
import requests
import numpy as np
from PIL import Image

# Using pre-trained MobileNet
torch_model = mobilenet_v2(pretrained=True)
torch_model.eval()

# Step 1: Trace model
input_shape = (1, 3, 224, 224)
example_input = torch.rand(input_shape)
traced_torch_model = torch.jit.trace(torch_model, example_input)

# Step 2: Compile model for Xiaomi 12
compile_job = hub.submit_compile_job(
    model=traced_torch_model,
    device=hub.Device("Xiaomi 12"),
    input_specs=dict(image=input_shape),
)

# Step 3: Profile on cloud-hosted device (Xiaomi 12)
target_model = compile_job.get_target_model()
profile_job = hub.submit_profile_job(
    model=target_model,
    device=hub.Device("Xiaomi 12"),
)

# Step 4: Run inference on cloud-hosted device
sample_image_url = (
    "https://qaihub-public-assets.s3.us-west-2.amazonaws.com/apidoc/input_image1.jpg"
)
response = requests.get(sample_image_url, stream=True)
response.raw.decode_content = True
image = Image.open(response.raw).resize((224, 224))
input_array = np.expand_dims(
    np.transpose(np.array(image, dtype=np.float32) / 255.0, (2, 0, 1)), axis=0
)

# Run inference using the on-device model on the input image
inference_job = hub.submit_inference_job(
    model=target_model,
    device=hub.Device("Xiaomi 12"),
    inputs=dict(image=[input_array]),
)
on_device_output = inference_job.download_output_data()

# Step 5: Post-processing the on-device output
output_name = list(on_device_output.keys())[0]
out = on_device_output[output_name][0]
on_device_probabilities = np.exp(out) / np.sum(np.exp(out), axis=1)

# Read the class labels for imagenet
sample_classes = "https://qaihub-public-assets.s3.us-west-2.amazonaws.com/apidoc/imagenet_classes.txt"
response = requests.get(sample_classes, stream=True)
response.raw.decode_content = True
categories = [str(s.strip()) for s in response.raw]

# Print top five predictions for the on-device model
print("Top-5 On-Device predictions:")
top5_classes = np.argsort(on_device_probabilities[0], axis=0)[-5:]
for c in reversed(top5_classes):
    print(f"{c} {categories[c]:20s} {on_device_probabilities[0][c]:>6.1%}")

# Step 6: Download model
target_model = compile_job.get_target_model()
target_model.download("mobilenet_v2.tflite")

Uploading model: 100%|██████████| 13.9M/13.9M [00:00<00:00, 16.7MB/s]


Scheduled compile job (jogk0lnw5) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jogk0lnw5/

Waiting for compile job (jogk0lnw5) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          
Scheduled profiling job (jn5q17kng) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jn5q17kng/



Uploading dataset: 100%|██████████| 150k/150k [00:00<00:00, 434kB/s]


Scheduled inference job (j1gl80zjp) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/j1gl80zjp/

Waiting for inference job (j1gl80zjp) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          


dataset-dv958vlz9.h5: 100%|██████████| 14.5k/14.5k [00:00<00:00, 4.82MB/s]


Top-5 On-Device predictions:
968 b'cup'                70.9%
504 b'coffee mug'         16.6%
967 b'espresso'            7.9%
809 b'soup bowl'           1.4%
659 b'mixing bowl'         1.3%


job_jogk0lnw5_optimized_tflite_mkngo1w0q.tflite: 100%|██████████| 13.3M/13.3M [00:00<00:00, 26.1MB/s]


'mobilenet_v2.tflite'